In [ ]:
"""
Backup code to initialize on Kaggle for the RSNA Intracranial Aneurysm Detection dataset.

This script allows:
- Loading DICOM volumes from the dataset
- Navigating through volumes using previous/next buttons
- Viewing individual slices with a slider
- Displaying metadata like Series UID and Modality

Intended for use in Kaggle notebooks as a quick start / backup.
"""

import matplotlib.pyplot as plt
import ipywidgets as widgets
from IPython.display import display, clear_output

# Global variables
current_volume = None
current_index = 1562  # test index = 3710, 3703

current_text = ""
current_indices = None

# Buttons to navigate previous and next
btn_prev = widgets.Button(description="<< Anterior")
btn_next = widgets.Button(description="Próximo >>")

# Label to display the current index
label_index = widgets.Label()

# Slider to select the slice
slice_slider = widgets.IntSlider(
    value=0, min=0, max=0, step=1, description='Slice:', continuous_update=False
)

# Output widget to display the image and information
output = widgets.Output()

def load_volume_and_update(index):
    """
    Load the DICOM volume for the given index and update the slider, label, and display.
    """
    global current_volume, current_index, current_text, current_indices, current_series_data, current_vol_raw
    current_index = index
    uid = train_df.iloc[index]["SeriesInstanceUID"]
    mod = train_df.iloc[index]["Modality"]
    
    # Load the volume and series data
    current_volume, current_indices, current_series_data = load_dicom(
        slice_size=config.slice_size,
        series_uid=uid,
        series_mapping_df=series_mapping_df,
        image_size=config.img_size,
        use_win_default=False
    )
    
    # Update the slider maximum and reset to first slice
    slice_slider.max = current_volume.shape[0] - 1
    slice_slider.value = 0
    
    # Update label with current index and modality
    label_index.value = f"Index atual: {current_index} / {len(train_df)-1} - Modality: {mod}"
    
    # Display the first slice
    with output:
        clear_output(wait=True)
        show_slice(0)

def show_slice(slice_idx):
    """
    Display a single slice from the current volume.
    """
    plt.figure(figsize=(6, 6))
    plt.imshow(current_volume[slice_idx], cmap='gray')
    plt.title(f"Slice {slice_idx} / {current_volume.shape[0] - 1} | {current_text} | {current_indices}")
    plt.axis('off')
    plt.show()

def on_slice_change(change):
    """
    Callback function when the slice slider value changes.
    """
    if change['name'] == 'value' and change['type'] == 'change':
        with output:
            clear_output(wait=True)
            show_slice(change['new'])

def on_prev_clicked(b):
    """
    Callback for the 'previous' button to load the previous volume.
    """
    global current_index
    if current_index > 0:
        load_volume_and_update(current_index - 1)

def on_next_clicked(b):
    """
    Callback for the 'next' button to load the next volume.
    """
    global current_index
    if current_index < len(train_df) - 1:
        load_volume_and_update(current_index + 1)

# Link events
slice_slider.observe(on_slice_change)
btn_prev.on_click(on_prev_clicked)
btn_next.on_click(on_next_clicked)

# Initialize display with the current index
load_volume_and_update(current_index)

# Layout the controls and output
controls = widgets.HBox([btn_prev, label_index, btn_next])
display(widgets.VBox([controls, slice_slider, output]))
